In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import plotly.express as px 
import math
import scipy.stats as stats
from datetime import datetime
import matplotlib.pyplot as plt
! pip install plotly
import plotly
! pip install cufflinks
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

SuperData = pd.read_csv("../../../../data/stage_I/superDataset.csv")

#### extract and seprate data like cases and deaths

In [4]:
TexasData = SuperData.loc[(SuperData["State"] == "TX") & (SuperData["countyFIPS"] != 0)]
TexasIdentifiers = TexasData.iloc[:,:5]
TexasPop = TexasIdentifiers["population"]
TexasCases = TexasData.filter(regex='cases$',axis=1)
TexasDeaths = TexasData.filter(regex='deaths$',axis=1)

In [5]:
#clean NAN and negatives from data
TexasCases = TexasCases.diff(axis=1)
TexasCases = TexasCases.replace(np.nan, 0)
TexasCases[TexasCases < 0] = 0

TexasDeaths = TexasDeaths.diff(axis=1)
TexasDeaths = TexasDeaths.replace(np.nan, 0)
TexasDeaths[TexasDeaths < 0] = 0

#### 

In [6]:
#function to normalize data, since 
def norm(x, population):
    return (x/population)*100000

#normalize Texas cases
count=0
counties = TexasPop.index
for county in counties:
    population = TexasPop.index[count]
    TexasCases.iloc[count] = TexasCases.iloc[count].apply(norm, args=(population,))
    count +=1
TexasCases = TexasCases.round(0).astype(int)

#normalize Texas deaths
count=0
counties = TexasPop.index
for county in counties:
    population = TexasPop.index[count]
    TexasDeaths.iloc[count] = TexasDeaths.iloc[count].apply(norm, args=(population,))
    count +=1
TexasDeaths = TexasDeaths.round(0).astype(int)

In [7]:
#create weekly data
num_days = len(TexasCases.columns)
num_weeks = int(num_days/7)
TexasWeeklyCases = pd.DataFrame()
for week in range(0, num_weeks):
    start=week*7
    end=start+6
    column_name = "cases_week_" + str(week+1)
    TexasWeeklyCases[column_name] = TexasCases.iloc[:,start:end].mean(axis=1)
TexasWeeklyCases = TexasWeeklyCases.round(0).astype(int)

num_days = len(TexasDeaths.columns)
num_weeks = int(num_days/7)
TexasWeeklyDeaths = pd.DataFrame()
for week in range(0, num_weeks):
    start=week*7
    end=start+6
    column_name = "cases_week_" + str(week+1)
    TexasWeeklyDeaths[column_name] = TexasDeaths.iloc[:,start:end].mean(axis=1)
TexasWeeklyDeaths = TexasWeeklyDeaths.round(0).astype(int)

In [8]:
#merge back data with idenifers
TexasWeelyDeathsFin = pd.merge(TexasIdentifiers, TexasWeeklyDeaths, left_index=True, right_index=True)
TexasWeelyCasesFin = pd.merge(TexasIdentifiers, TexasWeeklyCases, left_index=True, right_index=True)

In [9]:
TexasDWF = TexasWeelyDeathsFin.groupby(['State']).sum()
TexasCWF = TexasWeelyCasesFin.groupby(['State']).sum()

In [10]:
#generate weekly stats
meanTXCases = TexasCWF.mean(axis=1)
medianTXCases = TexasCWF.median(axis=1)
modeTXCases = TexasCWF.mode(axis=1)

meanTXDeaths = TexasDWF.mean(axis=1)
medianTXDeaths = TexasDWF.median(axis=1)
modeTXDeaths = TexasDWF.mode(axis=1)

### Member Task 1 - 1 - Weekly Stats for TX

In [11]:
print(meanTXDeaths,medianTXDeaths,modeTXDeaths)

State
TX    658821.285714
dtype: float64 State
TX    3365.0
dtype: float64        0
State   
TX     0


In [12]:
print(meanTXCases,medianTXCases,modeTXCases)

State
TX    880862.571429
dtype: float64 State
TX    182154.0
dtype: float64        0
State   
TX     0


#### Everyhting that was done above for Texas is done for three more states splitting, normalizing, and creating weekly data

##### NC data

In [13]:
#picking three states to comapre

#Taking data from NC spliting it, cleaning it, normailizing it, adding weeks, summing and grenrating weekly stats
NCData = SuperData.loc[(SuperData["State"] == "NC") & (SuperData["countyFIPS"] != 0)]
NCIdentifiers = NCData.iloc[:,:5]
NCPop = NCIdentifiers["population"]
NCCases = NCData.filter(regex='cases$',axis=1)
NCDeaths = NCData.filter(regex='deaths$',axis=1)

NCCases = NCCases.diff(axis=1)
NCCases = NCCases.replace(np.nan, 0)
NCCases[NCCases < 0] = 0

NCDeaths = NCDeaths.diff(axis=1)
NCDeaths = NCDeaths.replace(np.nan, 0)
NCDeaths[NCDeaths < 0] = 0

count=0
counties = NCPop.index
for county in counties:
    population = NCPop.index[count]
    NCCases.iloc[count] = NCCases.iloc[count].apply(norm, args=(population,))
    count +=1
NCCases = NCCases.round(0).astype(int)

count=0
counties = NCPop.index
for county in counties:
    population = NCPop.index[count]
    NCDeaths.iloc[count] = NCDeaths.iloc[count].apply(norm, args=(population,))
    count +=1
NCDeaths = NCDeaths.round(0).astype(int)

#ws
num_days = len(NCCases.columns)
num_weeks = int(num_days/7)
NCWeeklyCases = pd.DataFrame()
for week in range(0, num_weeks):
    start=week*7
    end=start+6
    column_name = "cases_week_" + str(week+1)
    NCWeeklyCases[column_name] = NCCases.iloc[:,start:end].mean(axis=1)
NCWeeklyCases = NCWeeklyCases.round(0).astype(int)

num_days = len(NCDeaths.columns)
num_weeks = int(num_days/7)
NCWeeklyDeaths = pd.DataFrame()
for week in range(0, num_weeks):
    start=week*7
    end=start+6
    column_name = "cases_week_" + str(week+1)
    NCWeeklyDeaths[column_name] = NCDeaths.iloc[:,start:end].mean(axis=1)
NCWeeklyDeaths = NCWeeklyDeaths.round(0).astype(int)

##### FL data

In [40]:
#Taking data from FL spliting it, cleaning it, normailizing it, adding weeks, summing and grenrating weekly stats
FLData = SuperData.loc[(SuperData["State"] == "FL") & (SuperData["countyFIPS"] != 0)]
FLIdentifiers = FLData.iloc[:,:5]
FLPop = FLIdentifiers["population"]
FLCases = FLData.filter(regex='cases$',axis=1)
FLDeaths = FLData.filter(regex='deaths$',axis=1)

FLCases = FLCases.diff(axis=1)
FLCases = FLCases.replace(np.nan, 0)
FLCases[FLCases < 0] = 0

FLDeaths = FLDeaths.diff(axis=1)
FLDeaths = FLDeaths.replace(np.nan, 0)
FLDeaths[FLDeaths < 0] = 0

count=0
counties = FLPop.index
for county in counties:
    population = FLPop.index[count]
    FLCases.iloc[count] = FLCases.iloc[count].apply(norm, args=(population,))
    count +=1
FLCases = FLCases.round(0).astype(int)

count=0
counties = FLPop.index
for county in counties:
    population = FLPop.index[count]
    FLDeaths.iloc[count] = FLDeaths.iloc[count].apply(norm, args=(population,))
    count +=1
FLDeaths = FLDeaths.round(0).astype(int)

num_days = len(FLCases.columns)
num_weeks = int(num_days/7)
FLWeeklyCases = pd.DataFrame()
for week in range(0, num_weeks):
    start=week*7
    end=start+6
    column_name = "cases_week_" + str(week+1)
    FLWeeklyCases[column_name] = FLCases.iloc[:,start:end].mean(axis=1)
FLWeeklyCases = FLWeeklyCases.round(0).astype(int)

num_days = len(FLDeaths.columns)
num_weeks = int(num_days/7)
FLWeeklyDeaths = pd.DataFrame()
for week in range(0, num_weeks):
    start=week*7
    end=start+6
    column_name = "cases_week_" + str(week+1)
    FLWeeklyDeaths[column_name] = FLDeaths.iloc[:,start:end].mean(axis=1)
FLWeeklyDeaths = FLWeeklyDeaths.round(0).astype(int)

##### CA data

In [41]:

#Taking data from CA spliting it, cleaning it, normailizing it, adding weeks, summing and grenrating weekly stats
CAData = SuperData.loc[(SuperData["State"] == "CA") & (SuperData["countyFIPS"] != 0)]
CAIdentifiers = CAData.iloc[:,:5]
CAPop = CAIdentifiers["population"]
CACases = CAData.filter(regex='cases$',axis=1)
CADeaths = CAData.filter(regex='deaths$',axis=1)

CACases = CACases.diff(axis=1)
CACases = CACases.replace(np.nan, 0)
CACases[CACases < 0] = 0

CADeaths = CADeaths.diff(axis=1)
CADeaths = CADeaths.replace(np.nan, 0)
CADeaths[CADeaths < 0] = 0

count=0
counties = CAPop.index
for county in counties:
    population = CAPop.index[count]
    CACases.iloc[count] = CACases.iloc[count].apply(norm, args=(population,))
    count +=1
CACases = CACases.round(0).astype(int)

count=0
counties = CAPop.index
for county in counties:
    population = CAPop.index[count]
    CADeaths.iloc[count] = CADeaths.iloc[count].apply(norm, args=(population,))
    count +=1
CADeaths = CADeaths.round(0).astype(int)

num_days = len(CACases.columns)
num_weeks = int(num_days/7)
CAWeeklyCases = pd.DataFrame()
for week in range(0, num_weeks):
    start=week*7
    end=start+6
    column_name = "cases_week_" + str(week+1)
    CAWeeklyCases[column_name] = CACases.iloc[:,start:end].mean(axis=1)
CAWeeklyCases = CAWeeklyCases.round(0).astype(int)

num_days = len(CADeaths.columns)
num_weeks = int(num_days/7)
CAWeeklyDeaths = pd.DataFrame()
for week in range(0, num_weeks):
    start=week*7
    end=start+6
    column_name = "cases_week_" + str(week+1)
    CAWeeklyDeaths[column_name] = CADeaths.iloc[:,start:end].mean(axis=1)
CAWeeklyDeaths = CAWeeklyDeaths.round(0).astype(int)

##### merege above data back with Identifiers coulmns for things like county name and population

In [14]:
NCWeelyDeathsFin = pd.merge(NCIdentifiers, NCWeeklyDeaths, left_index=True, right_index=True)
NCWeelyCasesFin = pd.merge(NCIdentifiers, NCWeeklyCases, left_index=True, right_index=True)

FLWeelyDeathsFin = pd.merge(FLIdentifiers, FLWeeklyDeaths, left_index=True, right_index=True)
FLWeelyCasesFin = pd.merge(FLIdentifiers, FLWeeklyCases, left_index=True, right_index=True)

CAWeelyDeathsFin = pd.merge(CAIdentifiers, CAWeeklyDeaths, left_index=True, right_index=True)
CAWeelyCasesFin = pd.merge(CAIdentifiers, CAWeeklyCases, left_index=True, right_index=True)

##### sum weekly states for states

In [15]:
NCDGS = NCWeelyDeathsFin.groupby(['State']).sum()
NCCGS = NCWeelyCasesFin.groupby(['State']).sum()

FLDGS = FLWeelyDeathsFin.groupby(['State']).sum()
FLCGS = FLWeelyCasesFin.groupby(['State']).sum()

CADGS = CAWeelyDeathsFin.groupby(['State']).sum()
CACGS = CAWeelyCasesFin.groupby(['State']).sum()

meanNCCases = NCCGS.mean(axis=1)
meanNCCases = NCCGS.median(axis=1)

meanNCDeaths = NCDGS.mean(axis=1)
medianNCDeaths = NCDGS.median(axis=1)

meanFLCases = FLCGS.mean(axis=1)
medianFLCases = FLCGS.median(axis=1)

meanFLDeaths = FLDGS.mean(axis=1)
medianFLDeaths = FLDGS.median(axis=1)

meanCACases = CACGS.mean(axis=1)
medianCACases = CACGS.median(axis=1)

meanCADeaths = CADGS.mean(axis=1)
medianCADeaths = CADGS.median(axis=1)

### Member Task 1 - 2 - Compare Weekly Statistics for Other States

In [16]:
meanNCCases, meanNCCases, meanNCDeaths, medianNCDeaths

(State
 NC    83381.0
 dtype: float64,
 State
 NC    83381.0
 dtype: float64,
 State
 NC    226780.714286
 dtype: float64,
 State
 NC    1145.0
 dtype: float64)

In [17]:
meanFLCases, meanFLCases, meanFLDeaths, medianFLDeaths

(State
 FL    1.598936e+06
 dtype: float64,
 State
 FL    1.598936e+06
 dtype: float64,
 State
 FL    373473.174603
 dtype: float64,
 State
 FL    20208.0
 dtype: float64)

In [18]:
meanCACases, meanCACases, meanCADeaths, medianCADeaths

(State
 CA    4.357795e+06
 dtype: float64,
 State
 CA    4.357795e+06
 dtype: float64,
 State
 CA    690799.539683
 dtype: float64,
 State
 CA    32419.0
 dtype: float64)

### Member Task 1 - 3

In [42]:
#identify top 5 states in TX
TexasDailyCasesFin = pd.merge(TexasIdentifiers, TexasCases, left_index=True, right_index=True)
TexasDailyDeathsFin = pd.merge(TexasIdentifiers, TexasDeaths, left_index=True, right_index=True)
TDCS = TexasDailyCasesFin.sort_values(by=['population'], ascending=False)
TDDS = TexasDailyDeathsFin.sort_values(by=['population'], ascending=False)
TDCS.drop(['population', 'StateFIPS', 'State','countyFIPS'], axis=1, inplace=True)
TDDS.drop(['population', 'StateFIPS', 'State','countyFIPS'], axis=1, inplace=True)
TDCSF = TDCS.head(5)
TDDSF = TDDS.head(5)

### Member Task 1 - 4 

#### From the top 5 counties a significant peak in cases can be seen towards the end of the year of 2020

In [20]:
TDCSF.T.iplot(kind='scatter', title='Texas: Daily Cases Top 5 Counties')

#### From the top 5 counties a significant peak deaths can be seen in month 8

In [21]:
TDDSF.T.iplot(kind='scatter', title='Texas: Daily Deaths Top 5 Counties')

In [22]:
TexasDWFT = TexasDWF.T
TexasDWFT.drop(['countyFIPS', 'StateFIPS', 'population'], axis=0, inplace=True)
TexasCWFT = TexasCWF.T
TexasCWFT.drop(['countyFIPS', 'StateFIPS', 'population'], axis=0, inplace=True)

#### From the weekly cases in Texas, a sharp increase can be seen on week 49 of 2020 which is around thanksgiving

In [23]:
TexasDailyCasesTotal = TexasWeelyCasesFin.groupby(['State']).sum()
TexasDailyCasesTotal.drop(['countyFIPS', 'StateFIPS', 'population'], axis=1, inplace=True)
tff = TexasDailyCasesTotal.T
tff.iplot(kind='scatter', title='Texas: Weekly Cases')

#### From the weekly deahts in Texas, a sharp increase can be seen on week 27 of 2020 and at the end of 2020 which can be explained by the holiday season

In [24]:
TexasDailyDeathsTotal = TexasWeelyDeathsFin.groupby(['State']).sum()
TexasDailyDeathsTotal.drop(['countyFIPS', 'StateFIPS', 'population'], axis=1, inplace=True)
tfff = TexasDailyDeathsTotal.T
tfff.iplot(kind='scatter', title='Texas: Weekly Deaths')

In [25]:
px.histogram(TDCSF.mean(axis=1))

In [26]:
px.histogram(TDDSF.mean(axis=1))

# Member Task 2

In [27]:
# Take previously normalized data and add a mean column that will be used to create a histogram, then remove all columns expect for country name and mean cases or deaths for that county
TexasDailyCasesFin['mean'] = TexasDailyCasesFin.mean(axis=1)
TexasDailyDeathsFin['mean'] = TexasDailyDeathsFin.mean(axis=1)
TXCountyCases = TexasDailyCasesFin[["County Name", "mean"]]
TXDeathsCases = TexasDailyDeathsFin[["County Name", "mean"]]

In [28]:
#plot the cases and deaths into a hisogram into bins to show by how much cases and deahts in counties are incresing by
TXCountyCasesPlot = px.histogram(TXCountyCases, x=TXCountyCases["mean"], nbins=15, labels= {'mean':'Daily New Cases Across Texas', 'count':'probability'})
TXDeathsCasesPlot = px.histogram(TXDeathsCases, x=TXDeathsCases["mean"], nbins=15, labels={'mean':'Daily New Deaths Across Texas', 'count':'probability'})

# Member Task 2 - 1

## Distribution of Cases
### The distribution for cases appears to be a geometric distribution, it starts out relatively high and goes down,  it has a postive skew kurtosis

In [29]:
TXCountyCasesPlot

## Distribution of Deaths
### The distribution for deaths appears to be a geometric distribution, it starts out relatively high and goes down, it has a postive skew kurtosis

In [30]:
TXDeathsCasesPlot

# Member Task 2 - 2

### Texas Cases Poission Distribution 

In [31]:
#the mu value is the perviouly calcauted mean in task 1 of the member task
mu = 903585
random_sample = stats.poisson.rvs(mu=mu, size=1000)
poisson = px.histogram(random_sample)
poisson

### Texas Deaths Poission Distribution 

In [32]:
#the mu value is the perviouly calcauted mean in task 1 of the member task
mu = 680204
ransam = stats.poisson.rvs(mu=mu, size=1000)
poisson = px.histogram(ransam,)
poisson

# Member Task 2 - 2

In [33]:
#the mu value is the perviouly calcauted mean in task 1 of the member task
ncMu = 7
flMu = 10
caMu = 18
ransam = stats.poisson.rvs(mu=ncMu, size=1000)
ncPoisson = px.histogram(ransam,)
ransam = stats.poisson.rvs(mu=flMu, size=1000)
flPoisson = px.histogram(ransam,)
ransam = stats.poisson.rvs(mu=flMu, size=1000)
caPoisson = px.histogram(ransam,)

In [34]:
ncPoisson

In [35]:
flPoisson

In [36]:
caPoisson

# Member Task 2 - 4

In [37]:
HospitalBedData = pd.read_csv("../../../../data/stage_I/Definitive_Healthcare _USA_Hospital_Beds.csv")
TexasBedData = HospitalBedData.loc[(HospitalBedData["HQ_STATE"] == "TX")]
TBD = TexasBedData.drop(['X','Y','OBJECTID','HQ_ZIP_CODE','FIPS','HQ_ADDRESS1','COUNTY_NAME','STATE_FIPS','CNTY_FIPS','HOSPITAL_NAME','HOSPITAL_TYPE','HQ_ADDRESS','HQ_CITY'], axis=1)
TBDM = TBD.groupby(['HQ_STATE']).sum()
TBDM.head()

,NUM_LICENSED_BEDS,NUM_STAFFED_BEDS,NUM_ICU_BEDS,ADULT_ICU_BEDS,PEDI_ICU_BEDS,BED_UTILIZATION,AVG_VENTILATOR_USAGE,Potential_Increase_In_Bed_Capac
HQ_STATE,,,,,,,,
TX,81737.0,71982.0,8688,8688,3637.0,263.780372,1916.0,9755


In [39]:
TBD['NUM_LICENSED_BEDS'].corr(TBD['BED_UTILIZATION'])

0.33190753166967907

### The correlction between hospital bed data in the COVID data is both can be used off of each other to show how COIVD is affecting a community, less hospital beds and ventlatiors can be correlated to more deaths, less hospitals in general mean more cases and deaths in general

# Member Task 2 - 5

### Hospital Bed Hypothesis - The fewer hospital beds and ventilators available in a county the more deaths occur from COVID in that county as a result 